### 2.2 Types de données PostgreSQL -  Exemples de  Listes

In [1]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    database="demo",
    user="demo_owner",
    password="OODBMS")

c = conn.cursor()

<div style="font-size:120%" id="create">
Création d'une table comportant des champs liste :
</div>

In [2]:
sql = 'DROP TABLE IF EXISTS fibonacci;'
c.execute(sql)

sql = '''CREATE TABLE fibonacci (
    index    integer,
    value    integer,
    previous integer[2],
    sequence integer[]
);'''
c.execute(sql)
conn.commit()

<div style="font-size:120%" id="data">
Création de quelques données :
</div>

In [3]:
fibo = [0,1]
prev = [[0,0],[0,0]]
for n in range(2,7):
    fibo.append(fibo[n-2] + fibo[n-1])
    prev.append([fibo[n-2],fibo[n-1]])

<div style="font-size:120%" id="insert_sql">
Renseignement des données dans la base :
</div>

In [4]:
c.execute("TRUNCATE TABLE fibonacci;")

# SQL natif
sql = "INSERT INTO fibonacci VALUES ({},{},{},{});"
for n in range(7):
    previous = 'ARRAY['+','.join([str(p) for p in prev[n]])+']'
    sequence = 'ARRAY['+','.join([str(f) for f in fibo[:n+1]])+']'
    request = sql.format(n,fibo[n],previous,sequence)
    print(request)
    c.execute(request)
    
conn.commit()

INSERT INTO fibonacci VALUES (0,0,ARRAY[0,0],ARRAY[0]);
INSERT INTO fibonacci VALUES (1,1,ARRAY[0,0],ARRAY[0,1]);
INSERT INTO fibonacci VALUES (2,1,ARRAY[0,1],ARRAY[0,1,1]);
INSERT INTO fibonacci VALUES (3,2,ARRAY[1,1],ARRAY[0,1,1,2]);
INSERT INTO fibonacci VALUES (4,3,ARRAY[1,2],ARRAY[0,1,1,2,3]);
INSERT INTO fibonacci VALUES (5,5,ARRAY[2,3],ARRAY[0,1,1,2,3,5]);
INSERT INTO fibonacci VALUES (6,8,ARRAY[3,5],ARRAY[0,1,1,2,3,5,8]);


<div style="font-size:120%" id="insert_dbapi">
Méthode alternative avec des arguments à la DB-API:
</div>

In [5]:
c.execute("TRUNCATE TABLE fibonacci;")

sql = "INSERT INTO fibonacci VALUES (%s,%s,%s,%s);"
for n in range(7):
    args = (n,fibo[n],prev[n],fibo[:n+1])
    print(args)
    c.execute(sql,args)

(0, 0, [0, 0], [0])
(1, 1, [0, 0], [0, 1])
(2, 1, [0, 1], [0, 1, 1])
(3, 2, [1, 1], [0, 1, 1, 2])
(4, 3, [1, 2], [0, 1, 1, 2, 3])
(5, 5, [2, 3], [0, 1, 1, 2, 3, 5])
(6, 8, [3, 5], [0, 1, 1, 2, 3, 5, 8])


<div style="font-size:120%" id="select">
Requêtage des données :
</div>

In [6]:
c = conn.cursor()
sql = 'SELECT * FROM fibonacci WHERE index=6'
c.execute(sql)
print(c.fetchone())

(6, 8, [3, 5], [0, 1, 1, 2, 3, 5, 8])


In [7]:
c = conn.cursor()
sql = 'SELECT * FROM fibonacci WHERE sequence[5]=3'
c.execute(sql)
for d in c.fetchall(): print(d)

(4, 3, [1, 2], [0, 1, 1, 2, 3])
(5, 5, [2, 3], [0, 1, 1, 2, 3, 5])
(6, 8, [3, 5], [0, 1, 1, 2, 3, 5, 8])


In [8]:
conn.commit()